# Explore data
## Project: Cycling node network loop analysis

This notebook explores the input data set.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2024-01-24  
Last modified: 2024-01-26  

## To do

* Double-check cycle/edge lengths. For example 3-cycle east of Faxe
* If node-based analysis, then need to add cycle permutations?
* X Drop non-main nodes
* X Drop loops (they are really dangling links)
* X Find all simple cycles (bounded?-max length?) with networkX

## Parameters

In [ ]:
cycle_numnode_bound = 20  # From 30 it starts getting slow

## Imports

In [ ]:
import geopandas as gpd
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from functools import reduce

## Functions

In [ ]:
def NormalizeData(data):
    return list((data - np.min(data)) / (np.max(data) - np.min(data)))

def getLayout(G, nodes_id, nodes_coords):
    named_vertex_list = G.vs()["name"]
    layout = []
    for n in named_vertex_list:
        pos = nodes_id.index(n)
        layout.append(nodes_coords[pos])
    return layout

def plotCheck(G, nodes_id, nodes_coords):
    fig, ax = plt.subplots()
    layout = getLayout(G, nodes_id, nodes_coords)
    ig.plot(G, target=ax, vertex_size=6, layout=layout);
    
def getCycleLength(c):
    l = 0
    cl = len(c)
    for i in range(cl):
        l += Gnx[c[i%cl]][c[(i+1)%cl]]["weight"]
    return l

## Exploration

### Load data

In [ ]:
edges = gpd.read_file(r'../data/input/faxe/network/network_edges_no_parallel.gpkg')
nodes = gpd.read_file(r'../data/input/faxe/network/nodes_edges_parallel.gpkg')
# Set CRS
edges.set_crs('epsg:25832')
nodes.set_crs('epsg:25832');

In [ ]:
edges.head()

In [ ]:
nodes.head()

In [ ]:
nodes_id = list(nodes.id)
nodes_x = list(nodes.geometry.x)
nodes_y = list(nodes.geometry.y)
nodes_coords = list(zip(NormalizeData(nodes_x), NormalizeData(nodes_y)))

In [ ]:
# Rename length to weight for igraph
edges = edges.rename(columns={"length": "weight"})
# Drop unused columns
used_columns = {"u":(), "v":(), "weight":()}
for c_name, _ in edges.items():
    if c_name not in used_columns:
        del edges[c_name]

# Reorder columns
edges = edges[['u','v','weight']]

### Turn into igraph object

In [ ]:
G = ig.Graph.TupleList(edges.itertuples(index=False), directed=False, weights=True)

In [ ]:
G.summary()

In [ ]:
# Plot to double-check
plotCheck(G, nodes_id, nodes_coords)

### Drop self-loops

They are really dangling links which go outside the region, were mistakenly connected to themselves.

In [ ]:
G.simplify(multiple=True, loops=True, combine_edges=min);

In [ ]:
# Plot to double-check
plotCheck(G, nodes_id, nodes_coords)

### Drop dangling nodes

In [ ]:
# Source: https://codereview.stackexchange.com/questions/284246/deletion-of-nodes-of-degree-1-from-a-python-igraph-graph
vertices = {v for v in G.vs.select(_degree_le=1)}
needs_to_be_checked = set(vertices)
while needs_to_be_checked:
    vertex = needs_to_be_checked.pop()
    for n_vertex in vertex.neighbors():
        if n_vertex in vertices \
                or sum(1 for v in n_vertex.neighbors() if v not in vertices) > 1:
            continue
        vertices.add(n_vertex)
        needs_to_be_checked.add(n_vertex)
G.delete_vertices(vertices)

In [ ]:
# Plot to double-check
plotCheck(G, nodes_id, nodes_coords)

### Drop degree 2 nodes

This should include all non-ismain nodes.

In [ ]:
nodes_nonismain = nodes.loc[nodes['ismain'] == 0]
nodes_nonismain = nodes_nonismain['node_id'].to_list()
# Turn to dict for fast finding
nodes_nonismain = {nodes_nonismain[i]: True for i in range(len(nodes_nonismain))} 

In [ ]:
to_delete_ids = []

# Unclear how to select nodes in igraph by name, so let's iterate through them
for v in G.vs:
#     if v["name"] in nodes_nonismain and v.degree() == 2:
    if v.degree() == 2:
        # Remember node to delete
        to_delete_ids.append(v.index)
        # Add a new edge that combines the deleted ones
        sumoflengths = v.incident()[0].attributes()["weight"] + v.incident()[1].attributes()["weight"]
        G.add_edge(v.neighbors()[0].index, v.neighbors()[1].index, weight = sumoflengths)

G.delete_vertices(to_delete_ids)

# Re-simplify
G.simplify(multiple=True, loops=True, combine_edges=min);

In [ ]:
# Plot to double-check
plotCheck(G, nodes_id, nodes_coords)

### Get cycle basis

In [ ]:
# https://python.igraph.org/en/latest/api/igraph.GraphBase.html#fundamental_cycles
fcycles = {}
fcycles_lengths = []
cid = 0
for c in G.fundamental_cycles():
    # Add some statistics
    ws = [G.es(eid)['weight'] for eid in c]
    fcycles[cid] = {"edges": c, "length": sum(reduce(lambda a, b: a + b, ws))}
    fcycles_lengths.append(len(c))
    cid += 1

Getting all simple cycles has not yet been implemented in igraph, see:  
* https://github.com/igraph/igraph/issues/379  
* https://github.com/igraph/igraph/issues/1398  
Some potential progress here, but only for C, not Python:
* https://github.com/igraph/igraph/pull/2181

But they can be XORed through the cycle base.  

It has been implemented in networkX though: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.cycles.simple_cycles.html#networkx.algorithms.cycles.simple_cycles

### Get all simple cycles via nx

In [ ]:
Gnx = G.to_networkx()

In [ ]:
# # Sanity checks
# for (u, v, wt) in Gnx.edges.data('weight'):
#     print(f"({u}, {v}, {wt:.8})")

In [ ]:
cycle_numnode_bound = 3
allcycles_generator = nx.simple_cycles(Gnx, length_bound=cycle_numnode_bound)

In [ ]:
allcycles = {}
nodes_done = set()
numcycles = 0
for c in allcycles_generator:
    sourcenode = c[0]
    c_length = getCycleLength(c)
    numcycles += 1
    if sourcenode in nodes_done:
        allcycles[sourcenode]["cycles"].append(c)
        allcycles[sourcenode]["lengths"].append(c_length)
        allcycles[sourcenode]["numnodes"].append(len(c))
    else:
        allcycles[sourcenode] = {"cycles": [c], "lengths": [c_length], "numnodes": [len(c)]}
        nodes_done.add(sourcenode)
print("Found " + str(numcycles) + " cycles for length bound " + str(cycle_numnode_bound))

In [ ]:
# allcycles

In [ ]:
allcyclelengths = np.zeros(numcycles)
allcyclenumnodes = np.zeros(numcycles, dtype=int)
i = 0
for j in allcycles:
    l = len(allcycles[j]["lengths"])
    allcyclelengths[i:i+l] = allcycles[j]["lengths"]
    allcyclenumnodes[i:i+l] = allcycles[j]["numnodes"]
    i += l

In [ ]:
fig = plt.figure(figsize=(8, 3))
axes1 = fig.add_axes([0.1, 0.1, 0.35, 0.8])
axes2 = fig.add_axes([0.55, 0.1, 0.35, 0.8])

axes1.hist(allcyclelengths, density=True)
axes1.set_xlabel('Length [m]')
axes1.set_ylabel('Frequency')
axes1.set_title('Cycle lengths')

axes2.hist(allcyclenumnodes, density=True, bins=list(range(cycle_numnode_bound+1)))
axes2.set_xlabel('Nodes')
axes2.set_title('Nodes per cycle')
axes2.set_xlim([0, cycle_numnode_bound+0.5])

plt.text(cycle_numnode_bound/20,0.01, "Bound: " + str(cycle_numnode_bound))
plt.text(cycle_numnode_bound/20,0.04, "Cycles: " + str(numcycles));